In [ ]:
!pip install torch
!pip install -qqq transformers
!pip install -qqq langchain
!pip install -qqq tokenizers
!pip install -qqq optimum
!pip install -qqq auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
import re
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
def getAnswer(question):
  template = """
  <s>[INST] <<SYS>>
  Take a deep breathe and work on this step by step.
  You are a SQL developer that wants to extract information from a database using SQL queries. Your task is to generate SQL queries based on the input natural language. Your queries should follow the SQL standard and be compatible with the company's database management system. Ensure that your queries are structured logically, efficiently, and properly formatted.Do not explain.Provide the SQL query only.
  It is important for me, so please generate an efficient syntactically correct SQL query.
  <</SYS>>


  {text} [/INST]
  """



  prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  ans=chain.run(question)
  sql_query = re.search(r'SELECT[^;]+;', ans, re.DOTALL)

  if sql_query:
    sql_query = sql_query.group().strip()
  else:
    sql_query="Please repharse your natural language question"

  return sql_query

In [ ]:
question="Retrieve all products."
result = getAnswer(question)
print(result)

SELECT * FROM products;


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.6 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

In [ ]:
class Req(BaseModel):
  question: str


In [ ]:
@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}


@app.post('/predict')
def getAnswer(req: Req):
  template = """https://8dcb-35-226-244-64.ngrok-free.app/
  <s>[INST] <<SYS>>
  Take a deep breathe and work on this step by step.
  You are a SQL developer that wants to extract information from a database using SQL queries. Your task is to generate SQL queries based on the input natural language. Your queries should follow the SQL standard and be compatible with the company's database management system. Ensure that your queries are structured logically, efficiently, and properly formatted.Do not explain.Provide the SQL query only.
  It is important for me, so please generate an efficient syntactically correct SQL query.
  <</SYS>>


  {text} [/INST]
  """



  prompt = PromptTemplate(
      input_variables=["text"],
      template=template,
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  ans=chain.run(req.question)
  sql_query = re.search(r'SELECT[^;]+;', ans, re.DOTALL)

  if sql_query:
    sql_query = sql_query.group().strip()
  else:
    sql_query="Please repharse your natural language question"


  return {"role": "assistant", "query": sql_query}

# Setup Ngrok

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

authToken = "2fKCVnCZQrkWbfEkDOIHTYtkf6N_5P4dDWnqADtucroPUHXuU"
ngrok.set_auth_token(authToken)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Exception in thread Thread-11 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)
INFO:     Started server process [1528]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://01e1-34-173-89-185.ngrok-free.app
INFO:     197.36.200.244:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     197.36.200.244:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     197.36.200.244:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     197.36.200.244:0 - "POST /predict HTTP/1.1" 200 OK
